In [38]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2

# =====================================================
# 1. YARDIMCI FONKSİYONLAR (AYNI)
# =====================================================

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    return 2 * R * atan2(sqrt(a), sqrt(1 - a))

def route_distance(route):
    return sum(
        haversine(
            route[i-1]["lat"], route[i-1]["lon"],
            route[i]["lat"], route[i]["lon"]
        )
        for i in range(1, len(route))
    )

def nearest_neighbor(points):
    if len(points) <= 1:
        return points
    unvisited = points.copy()
    route = [unvisited.pop(0)]
    while unvisited:
        last = route[-1]
        next_point = min(
            unvisited,
            key=lambda p: haversine(last["lat"], last["lon"], p["lat"], p["lon"])
        )
        route.append(next_point)
        unvisited.remove(next_point)
    return route

def two_opt(route):
    improved = True
    while improved:
        improved = False
        for i in range(1, len(route) - 2):
            for j in range(i + 1, len(route)):
                if j - i == 1:
                    continue
                new_route = route[:i] + route[i:j][::-1] + route[j:]
                if route_distance(new_route) < route_distance(route):
                    route = new_route
                    improved = True
    return route

# =====================================================
# 2. VERİYİ OKU
# =====================================================

containers = pd.read_csv("container_candidates.csv")

# =====================================================
# 3. TEMİZLİK (MINIMUM)
# =====================================================

containers["date"] = pd.to_datetime(containers["date"], errors="coerce").dt.date.astype(str)

# numeric lat/lon
containers["lat"] = pd.to_numeric(containers["lat"], errors="coerce")
containers["lon"] = pd.to_numeric(containers["lon"], errors="coerce")
containers = containers.dropna(subset=["lat", "lon", "date", "vehicle_id"])

# vehicle_type yoksa koru
if "vehicle_type" not in containers.columns:
    containers["vehicle_type"] = "UNKNOWN"

# score yoksa türet (yoksa sıralama için 0 basar)
if "score" not in containers.columns:
    containers["score"] = 0.0
else:
    containers["score"] = pd.to_numeric(containers["score"], errors="coerce").fillna(0.0)

# duration yoksa 0 yap (tekilleştirmede yedek kriter)
if "duration_min" not in containers.columns:
    containers["duration_min"] = 0.0
else:
    containers["duration_min"] = pd.to_numeric(containers["duration_min"], errors="coerce").fillna(0.0)

# =====================================================
# 3.1 TOP-K (vehicle-day bazında)
# =====================================================

TOP_K_PER_VEHICLE_DAY = 80  # istersen 50/100 yap
containers = (
    containers.sort_values(["date", "vehicle_id", "score", "duration_min"],
                           ascending=[True, True, False, False])
    .groupby(["date", "vehicle_id"], as_index=False)
    .head(TOP_K_PER_VEHICLE_DAY)
)

# =====================================================
# 3.2 YAKIN NOKTALARI TEKİLLEŞTİR (GRID ~ metre)
# =====================================================

GRID_METERS = 25  # 15-30m genelde iyi. GPS sapması yüksekse 30m yap.

def add_grid_id(df: pd.DataFrame, grid_m: float) -> pd.DataFrame:
    df = df.copy()
    # 1 derece lat ~ 111_320 m
    lat_step = grid_m / 111_320.0
    # lon adımı enlemle değişir: 1 derece lon ~ 111_320*cos(lat)
    lat_rad = np.deg2rad(df["lat"].astype(float))
    lon_step = grid_m / (111_320.0 * np.cos(lat_rad).clip(lower=0.2))  # kutup koruması

    df["grid_lat"] = np.floor(df["lat"] / lat_step).astype("int64")
    df["grid_lon"] = np.floor(df["lon"] / lon_step).astype("int64")

    # vehicle-day içinde tekilleştireceğimiz anahtar
    df["grid_id"] = df["grid_lat"].astype(str) + "_" + df["grid_lon"].astype(str)
    return df

containers = add_grid_id(containers, GRID_METERS)

# Aynı (date, vehicle_id, grid_id) içinde EN iyi kaydı tut:
# önce score, sonra duration_min büyük olan kazansın.
containers = (
    containers.sort_values(["date", "vehicle_id", "grid_id", "score", "duration_min"],
                           ascending=[True, True, True, False, False])
    .groupby(["date", "vehicle_id", "grid_id"], as_index=False)
    .head(1)
)

# =====================================================
# 4. ROTA OPTİMİZASYONU
# =====================================================

routes = []
vehicle_summary = []

for (date, vehicle_id), group in containers.groupby(["date", "vehicle_id"]):

    points = [
        {
            "Mahalle": (r["aciklama"] if "aciklama" in group.columns else "CONTAINER"),
            "lat": r["lat"],
            "lon": r["lon"]
        }
        for _, r in group.iterrows()
    ]

    if len(points) == 0:
        continue

    route = two_opt(nearest_neighbor(points))
    total_km = route_distance(route)

    prev = None
    for idx, stop in enumerate(route):
        dist_prev = 0 if prev is None else haversine(
            prev["lat"], prev["lon"], stop["lat"], stop["lon"]
        )
        routes.append({
            "date": date,
            "vehicle_id": vehicle_id,
            "vehicle_type": group["vehicle_type"].iloc[0],
            "stop_order": idx + 1,
            "Mahalle": stop["Mahalle"],   # burada "container açıklaması" gibi düşün
            "latitude": stop["lat"],
            "longitude": stop["lon"],
            "distance_from_prev_km": round(dist_prev, 3)
        })
        prev = stop

    vehicle_summary.append({
        "date": date,
        "vehicle_id": vehicle_id,
        "vehicle_type": group["vehicle_type"].iloc[0],
        "stops": len(route),
        "total_distance_km": round(total_km, 2)
    })

# =====================================================
# 5. KAYDET
# =====================================================

pd.DataFrame(routes).to_csv("phase5_routes.csv", index=False)
pd.DataFrame(vehicle_summary).to_csv("phase5_vehicle_summary.csv", index=False)

print("✅ FAZ 5 (CONTAINER + TEKİLLEŞTİRME) TAMAMLANDI")
print("-> phase5_routes.csv")
print("-> phase5_vehicle_summary.csv")
print(f"Tekilleştirme grid: {GRID_METERS}m, TOP_K: {TOP_K_PER_VEHICLE_DAY}")


✅ FAZ 5 (CONTAINER + TEKİLLEŞTİRME) TAMAMLANDI
-> phase5_routes.csv
-> phase5_vehicle_summary.csv
Tekilleştirme grid: 25m, TOP_K: 80


In [11]:
import pandas as pd

IN_FILE  = "phase4_assignments.csv"
OUT_FILE = "phase4_assignments_addrfmt.csv"   # yeni dosya

def to_address_format(mahalle: str) -> str:
    if pd.isna(mahalle):
        return mahalle

    m = str(mahalle).strip()

    # Tamamı büyükse sorun değil; önce normalize edelim
    m_upper = m.upper()

    # Sondaki "MAHALLESİ" (veya olası varyantlar) kaldır
    for suf in [" MAHALLESİ", " MAH.", " MAH", " MH.", " MH"]:
        if m_upper.endswith(suf):
            m_upper = m_upper[: -len(suf)].strip()
            break

    # Title Case (kelimelerin ilk harfi büyük)
    m_title = m_upper.title()

    # Sonuna " Mh." ekle
    return f"{m_title} Mh."

# Oku
df = pd.read_csv(IN_FILE)

# Kolon kontrol
if "Mahalle" not in df.columns:
    raise ValueError(f"{IN_FILE} içinde 'Mahalle' kolonu yok. Kolonlar: {list(df.columns)}")

# Dönüştür (Mahalle kolonunu direkt overwrite ediyoruz)
df["Mahalle"] = df["Mahalle"].apply(to_address_format)

# Kaydet
df.to_csv(OUT_FILE, index=False, encoding="utf-8-sig")

print("OK ->", OUT_FILE)
print("Örnek 10 mahalle:")
print(df["Mahalle"].dropna().head(10).to_string(index=False))

OK -> phase4_assignments_addrfmt.csv
Örnek 10 mahalle:
 Görükle Mh.
    Çali Mh.
 Üçevler Mh.
 Üçevler Mh.
 Üçevler Mh.
 Üçevler Mh.
Esentepe Mh.
Esentepe Mh.
Esentepe Mh.
Esentepe Mh.


In [15]:
import pandas as pd

# =========================
# Türkçe -> İngilizce karakter dönüşümü
# =========================
TR_TO_ENG = str.maketrans({
    "ç": "c", "Ç": "C",
    "ğ": "g", "Ğ": "G",
    "ı": "i", "I": "I", "İ": "I",
    "ö": "o", "Ö": "O",
    "ş": "s", "Ş": "S",
    "ü": "u", "Ü": "U"
})

def tr_to_eng(s):
    if pd.isna(s):
        return s
    return str(s).translate(TR_TO_ENG)

# =========================
# 1) address_data.csv -> address_data_eng.csv
#    neighborhood kolonu Türkçe karakterlerden arındırılacak
# =========================
addr = pd.read_csv("address_data.csv", low_memory=False)

if "neighborhood" not in addr.columns:
    raise ValueError(f"address_data.csv içinde 'neighborhood' kolonu yok. Kolonlar: {list(addr.columns)}")

addr["neighborhood"] = addr["neighborhood"].apply(tr_to_eng)

addr.to_csv("address_data_eng.csv", index=False, encoding="utf-8-sig")
print("OK -> address_data_eng.csv kaydedildi")

# =========================
# 2) phase4_assignments_addrfmt.csv -> phase4_assignments_eng.csv
#    Mahalle kolonu Türkçe karakterlerden arındırılacak
# =========================
ass = pd.read_csv("phase4_assignments_addrfmt.csv", low_memory=False)

if "Mahalle" not in ass.columns:
    raise ValueError(f"phase4_assignments_addrfmt.csv içinde 'Mahalle' kolonu yok. Kolonlar: {list(ass.columns)}")

ass["Mahalle"] = ass["Mahalle"].apply(tr_to_eng)

ass.to_csv("phase4_assignments_eng.csv", index=False, encoding="utf-8-sig")
print("OK -> phase4_assignments_eng.csv kaydedildi")


OK -> address_data_eng.csv kaydedildi
OK -> phase4_assignments_eng.csv kaydedildi


In [27]:
import pandas as pd
import re
import unicodedata
from difflib import get_close_matches

# =========================
# 0) DOSYALAR
# =========================
A_FILE = "phase4_assignments_eng.csv"   # veya phase4_assignments_addrfmt.csv hangisi gerçekse
B_FILE = "address_data_eng.csv"         # address_data.csv'den ürettiğin ingilizce karakterli dosya

A_COL  = "Mahalle"
B_COL  = "neighborhood"

OUT_FIXED = "phase4_assignments_fixed.csv"
OUT_MAP   = "phase5_neighborhood_mapping_suggestions.csv"

# =========================
# 1) KANONİKLEŞTİRME
# =========================
def canon(s: str) -> str:
    if pd.isna(s):
        return ""
    s = str(s)

    # Unicode normalize (farklı "i/ı", farklı nokta, NBSP vs yakalamak için)
    s = unicodedata.normalize("NFKC", s)

    # NBSP ve görünmez boşlukları normal space yap
    s = s.replace("\u00A0", " ").replace("\u200B", "").replace("\ufeff", "")

    # Trim + lower
    s = s.strip().lower()

    # Birden fazla boşluğu teke indir
    s = re.sub(r"\s+", " ", s)

    # Sonekleri standardize et: "mahallesi" / "mah." / "mh" / "mh." -> "mh"
    # (İstersen "mh." istiyorsan en sonda tekrar nokta ekleyebiliriz)
    s = re.sub(r"\bmahallesi\b", "mh", s)
    s = re.sub(r"\bmah\.\b", "mh", s)
    s = re.sub(r"\bmah\b", "mh", s)
    s = re.sub(r"\bmh\.\b", "mh", s)
    s = re.sub(r"\bmh\b", "mh", s)

    # Noktalama işaretlerini temizle (sonradan istersek geri ekleriz)
    s = re.sub(r"[.\-_/']", " ", s)
    s = re.sub(r"\s+", " ", s).strip()

    return s

def show_unicode_debug(label, s, max_chars=120):
    s = "" if pd.isna(s) else str(s)
    sn = unicodedata.normalize("NFKC", s)
    print(f"\n--- {label} ---")
    print("RAW repr :", repr(s[:max_chars]))
    print("NFKC repr:", repr(sn[:max_chars]))
    # ilk 80 karakterin codepointlerini göster
    cps = []
    for ch in sn[:80]:
        cps.append(f"U+{ord(ch):04X}({unicodedata.name(ch, 'UNKNOWN')})")
    print("CODEPOINTS:", " ".join(cps))

# =========================
# 2) OKU
# =========================
a = pd.read_csv(A_FILE, low_memory=False)
b = pd.read_csv(B_FILE, low_memory=False)

if A_COL not in a.columns:
    raise ValueError(f"{A_FILE} içinde '{A_COL}' kolonu yok. Kolonlar: {list(a.columns)}")
if B_COL not in b.columns:
    raise ValueError(f"{B_FILE} içinde '{B_COL}' kolonu yok. Kolonlar: {list(b.columns)}")

a["k"] = a[A_COL].apply(canon)
b["k"] = b[B_COL].apply(canon)

# Address tarafında koordinatlar (senin sistemin için)
# Eğer street_latitude/street_longitude yoksa burayı kendi kolonlarına göre düzelt.
LAT_COL = "street_latitude"
LON_COL = "street_longitude"
for c in [LAT_COL, LON_COL]:
    if c not in b.columns:
        raise ValueError(f"{B_FILE} içinde '{c}' kolonu yok. Kolonlar: {list(b.columns)}")

b[LAT_COL] = pd.to_numeric(b[LAT_COL], errors="coerce")
b[LON_COL] = pd.to_numeric(b[LON_COL], errors="coerce")
b_ok = b.dropna(subset=[LAT_COL, LON_COL]).copy()

# =========================
# 3) BİREBİR EŞLEŞME KONTROLÜ
# =========================
set_a = set(a["k"].dropna().unique())
set_b = set(b_ok["k"].dropna().unique())

only_in_a = sorted(set_a - set_b)
only_in_b = sorted(set_b - set_a)

print("A (phase4) unique canon mahalle:", len(set_a))
print("B (address, coord ok) unique canon mahalle:", len(set_b))
print("A'da olup B'de olmayan:", len(only_in_a))
print("B'de olup A'da olmayan:", len(only_in_b))

print("\nA'da olup B'de olmayan ilk 30:", only_in_a[:30])

# =========================
# 4) ŞÜPHELİ ÖRNEKLERDE GİZLİ KARAKTER ANALİZİ
#    (Senin örneğin: "cumhuriyet mh" gibi)
# =========================
probe = "cumhuriyet mh"
print("\nPROBE:", probe)
print("Phase4'te var mı?:", probe in set_a)
print("Address'te var mı?:", probe in set_b)

if probe not in set_a:
    # Phase4 tarafında 'cumhuriyet' içeren adayları göster
    cand_a = sorted([x for x in set_a if "cumhuriyet" in x])
    print("Phase4 'cumhuriyet' adayları:", cand_a[:20])
if probe in set_b:
    # Address'ten orijinal örneği bulup unicode debug yap
    raw_b = b_ok.loc[b_ok["k"] == probe, B_COL].iloc[0]
    show_unicode_debug("ADDRESS raw neighborhood örneği", raw_b)

# Eğer Phase4’te görünürde var diyorsan, Phase4’ten de yakalayıp debug edelim:
raw_a_rows = a.loc[a["k"].str.contains("cumhuriyet", na=False), A_COL].head(3).tolist()
if raw_a_rows:
    show_unicode_debug("PHASE4 raw Mahalle örneği", raw_a_rows[0])

# =========================
# 5) EŞLEŞMEYENLER İÇİN FUZZY ÖNERİ (KANONİK ÜZERİNDEN)
# =========================
# Not: difflib hızlı ve kurulu geliyor. RapidFuzz varsa daha iyi ama şart değil.
b_keys = sorted(set_b)

suggestions = []
for k in only_in_a:
    # en yakın 3 adayı öner
    close = get_close_matches(k, b_keys, n=3, cutoff=0.85)
    suggestions.append({
        "phase4_key": k,
        "suggested_address_keys": " | ".join(close) if close else ""
    })

map_df = pd.DataFrame(suggestions)
map_df.to_csv(OUT_MAP, index=False, encoding="utf-8-sig")
print(f"\nÖneri dosyası yazıldı: {OUT_MAP}")

# =========================
# 6) OTOMATİK DÜZELTME (SADECE %100 EMİN OLDUKLARINI UYGULA)
# =========================
# Burada otomatik uygulama için cutoff'u yüksek tutuyoruz.
auto_map = {}
for row in suggestions:
    k = row["phase4_key"]
    close = get_close_matches(k, b_keys, n=1, cutoff=0.92)
    if close:
        auto_map[k] = close[0]

print("Otomatik map'lenecek (yüksek güven):", len(auto_map))

# Phase4’te canon key'i bu mapping ile güncelle
a["k_fixed"] = a["k"].map(lambda x: auto_map.get(x, x))

# İstersen Phase4’ün görünen Mahalle yazısını da address formatına yaklaştıralım:
# Address'teki bir temsil değerini çekip yazalım (aynı key için en sık görünen)
rep = (
    b_ok.groupby("k")[B_COL]
    .agg(lambda s: s.value_counts().index[0])
    .to_dict()
)

# Eğer mapping sonrası k_fixed address tarafında varsa Mahalle'yi address temsil değeriyle değiştir
a[A_COL] = a["k_fixed"].map(lambda k: rep.get(k, None)).combine_first(a[A_COL])

# Kayıt
a.drop(columns=["k", "k_fixed"], inplace=True)
a.to_csv(OUT_FIXED, index=False, encoding="utf-8-sig")
print(f"\n✅ Düzeltilmiş phase4 kaydedildi: {OUT_FIXED}")

print("\nSONRA NE YAPACAKSIN?")
print(f"- Faz-5 kodunda assignments = pd.read_csv('{OUT_FIXED}') yap.")
print(f"- Address için zaten '{B_FILE}' kullan.")

A (phase4) unique canon mahalle: 61
B (address, coord ok) unique canon mahalle: 69
A'da olup B'de olmayan: 15
B'de olup A'da olmayan: 23

A'da olup B'de olmayan ilk 30: ['23 ni̇san mh', '30 agustos mh', 'ahmet yesevi̇ mh', 'alaadi̇nbey mh', 'altinsehi̇r mh', 'cumhuri̇yet mh', 'dagyeni̇ce mh', 'demi̇rci̇ mh', 'fethi̇ye mh', 'ihsani̇ye mh', 'inegazi̇ mh', 'irfani̇ye mh', 'kadri̇ye mh', 'maksempinar mh', 'mi̇nareli̇cavus mh']

PROBE: cumhuriyet mh
Phase4'te var mı?: False
Address'te var mı?: True
Phase4 'cumhuriyet' adayları: []

--- ADDRESS raw neighborhood örneği ---
RAW repr : 'Cumhuriyet Mh.'
NFKC repr: 'Cumhuriyet Mh.'
CODEPOINTS: U+0043(LATIN CAPITAL LETTER C) U+0075(LATIN SMALL LETTER U) U+006D(LATIN SMALL LETTER M) U+0068(LATIN SMALL LETTER H) U+0075(LATIN SMALL LETTER U) U+0072(LATIN SMALL LETTER R) U+0069(LATIN SMALL LETTER I) U+0079(LATIN SMALL LETTER Y) U+0065(LATIN SMALL LETTER E) U+0074(LATIN SMALL LETTER T) U+0020(SPACE) U+004D(LATIN CAPITAL LETTER M) U+0068(LATIN SMALL LET